## Stereo FM demodulator

In [1]:
import numpy as np


In [4]:
with open('fm941.bin', mode='rb') as file: # b is important -> binary
    x = file.read()

y = [(float(i)-127.5) for i in x]
y_re = y[0::2]
y_im = y[1::2]
len(y_re)

FileNotFoundError: [Errno 2] No such file or directory: 'fm941.bin'

In [3]:
import numpy as np

# Combine the real and imaginary parts to form the complex baseband signal
y_complex = np.array(y_re) + 1j*np.array(y_im)

### Step 1: Necessary Imports and FM Demodulation Function
First, ensure you have the necessary imports and reuse the FM demodulation function we defined previously for the mono signal.

In [4]:
import numpy as np
from scipy.signal import hilbert, butter, lfilter

def fm_demodulate(complex_signal, fm_deviation, sample_rate):
    """
    Demodulate a frequency modulated signal using the phase derivative method.
    """
    # Calculate the instantaneous phase of the complex signal
    instantaneous_phase = np.unwrap(np.angle(complex_signal))
    
    # Calculate the derivative of the phase
    instantaneous_freq = np.diff(instantaneous_phase)
    
    # Convert the derivative of the phase to a demodulated signal
    demodulated_signal = (instantaneous_freq / (2 * np.pi)) * sample_rate / fm_deviation
    
    # Append zero to maintain length consistency
    return np.append(demodulated_signal, 0)



### Step 2: Band-Pass Filters for Stereo Signal Components
You'll need band-pass filters to isolate the 19 kHz pilot tone and the 38 kHz stereo (L-R) signal.

In [5]:
def bandpass_filter(signal, lowcut, highcut, sample_rate, order=5):
    nyq = 0.5 * sample_rate
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, signal)
    return y


### Step 3: Extract and Decode Stereo Components
Now, extract the pilot tone to regenerate the 38 kHz signal and demodulate the stereo (L-R) component.

In [6]:
def lowpass_filter(signal, cutoff, sample_rate, order=5):
    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low')
    return lfilter(b, a, signal)

def stereo_decode(fm_signal, sample_rate):
    # Extract the pilot tone at 19 kHz
    pilot_tone = bandpass_filter(fm_signal, 18900, 19100, sample_rate, order=3)
    
    # Double the frequency of the pilot tone to generate the 38 kHz carrier
    carrier = np.cos(2 * np.pi * 2 * 19000 * np.arange(len(pilot_tone)) / sample_rate)
    
    # Extract the (L-R) component modulated around 38 kHz
    l_minus_r = bandpass_filter(fm_signal, 37000, 39000, sample_rate, order=3)
    
    # Demodulate the (L-R) component
    l_minus_r = l_minus_r * carrier
    
    # Low-pass filter to get the audio signal from the modulated carrier
    l_minus_r = lowpass_filter(l_minus_r, 15000, sample_rate, order=3)
    
    # Extract the mono (L+R) component
    l_plus_r = lowpass_filter(fm_signal, 15000, sample_rate, order=5)
    
    # Decode the original L and R channels
    left = (l_plus_r + l_minus_r) / 2
    right = (l_plus_r - l_minus_r) / 2
    
    return left, right


### Step 4: Use the Functions to Process Your Signal
Now you can use the fm_demodulate to get the FM baseband and then apply stereo_decode to separate the left and right audio channels.

In [7]:
# Example usage:
# Assume 'y_complex' is your I/Q data from the SDR
fm_signal = fm_demodulate(y_complex, 75000, 2400000)  # Example parameters
left, right = stereo_decode(fm_signal, 2400000)


## Make the stereo audio signal be the audio data

### Step 1: Normalize and Prepare the Audio Data
Before saving the audio, it's a good practice to normalize the audio to avoid clipping and to ensure consistent audio levels.

In [8]:
def normalize_audio(audio):
    # Normalize audio to prevent clipping
    audio = audio / np.max(np.abs(audio))
    return audio


Then apply this normalization to both the left and right channels:

In [9]:
left = normalize_audio(left)
right = normalize_audio(right)


### Step 2: Combine the Left and Right Channels
For scipy.io.wavfile.write, the stereo audio data should be in a two-dimensional array with two columns — one for each channel.

In [10]:
# Stack the left and right channels into a two-column array
stereo_audio = np.stack((left, right), axis=-1)

### Step 3: Write to a WAV File
Now, use scipy.io.wavfile.write to save the audio data to a file. Specify a suitable sample rate; ensure it's the same rate used for sampling your FM signal.

In [11]:
from scipy.io.wavfile import write

sample_rate = 2400000  # Adjust as per your SDR configuration
write('stereo_output.wav', sample_rate, stereo_audio.astype(np.float32))

### Step 4: Listening to the Audio
The saved WAV file, stereo_output.wav, can now be played using any standard audio player that supports WAV files. If the audio sounds too fast or too slow, verify that the sample rate specified in the write function matches the rate used during the signal processing steps.

## Full Example
Here's how the complete process might look when put together:

In [12]:
import numpy as np
from scipy.signal import hilbert, butter, lfilter
from scipy.io.wavfile import write

def bandpass_filter(signal, lowcut, highcut, sample_rate, order=5):
    nyq = 0.5 * sample_rate
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, signal)

def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio

def lowpass_filter(signal, cutoff, sample_rate, order=5):
    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low')
    return lfilter(b, a, signal)

def stereo_decode(fm_signal, sample_rate):
    # Extract the pilot tone at 19 kHz
    pilot_tone = bandpass_filter(fm_signal, 18900, 19100, sample_rate, order=3)
    
    # Double the frequency of the pilot tone to generate the 38 kHz carrier
    carrier = np.cos(2 * np.pi * 2 * 19000 * np.arange(len(pilot_tone)) / sample_rate)
    
    # Extract the (L-R) component modulated around 38 kHz
    l_minus_r = bandpass_filter(fm_signal, 37000, 39000, sample_rate, order=3)
    
    # Demodulate the (L-R) component
    l_minus_r = l_minus_r * carrier
    
    # Low-pass filter to get the audio signal from the modulated carrier
    l_minus_r = lowpass_filter(l_minus_r, 15000, sample_rate, order=3)
    
    # Extract the mono (L+R) component
    l_plus_r = lowpass_filter(fm_signal, 15000, sample_rate, order=5)
    
    # Decode the original L and R channels
    left = (l_plus_r + l_minus_r) / 2
    right = (l_plus_r - l_minus_r) / 2
    
    return left, right


# Example usage:
fm_signal = fm_demodulate(y_complex, 75000, 2400000)  # Adjust these parameters
left, right = stereo_decode(fm_signal, 2400000)  # Adjust sample rate accordingly

# Save the stereo audio to a WAV file
stereo_audio = np.stack((left, right), axis=-1)
write('stereo_output1.wav', 2400000, stereo_audio.astype(np.float32))
